# Define a Function to Create A Connected Graph with Different Levels of Connectivity

In [37]:
import random

def generate_connected_graph(n, extra_edges, max_weight):
    """
    Generate a connected undirected graph with n nodes.

    Parameters:
        n           : number of vertices (int)
        extra_edges : number of additional random edges (int)
        max_weight  : maximum weight for any edge (int)

    Returns:
        List of edges in (weight, u, v) format
    """
    edges = []
    connected = set([0])

    # Ensure connectivity by building a spanning tree
    for i in range(1, n):
        u = i
        v = random.choice(list(connected))
        weight = random.randint(1, max_weight)
        edge = (weight, min(u, v), max(u, v))
        edges.append(edge)
        connected.add(u)

    # Track existing edges for duplicate checking
    existing_edges = set((min(u, v), max(u, v)) for _, u, v in edges)

    # Precompute all possible edges
    all_possible_edges = {(min(u, v), max(u, v)) for u in range(n) for v in range(u + 1, n)}

    # Remove already existing edges
    remaining_edges = list(all_possible_edges - existing_edges)

    # If we request more extra edges than possible, limit it
    extra_edges = min(extra_edges, len(remaining_edges))

    # Sample from the remaining edges
    sampled_edges = random.sample(remaining_edges, extra_edges)

    # Add the sampled edges with random weights
    for u, v in sampled_edges:
        weight = random.randint(1, max_weight)
        edges.append((weight, u, v))

    return edges


# Kruskal's Algorithm Implementation

In [2]:
from Sorting import BucketSort, QuickSort

def KruskalsAlgorithm(edges, n, m,  compress = True, Bucket = True):
    """
    Implements Kruskal's algorithm to find the Minimum Spanning Tree.
    
    Parameters:
        edges: List of edges in {(u, v): weight} format
        n: Number of vertices in the graph
    
    Returns:
        mst_edges: List of edges in the Minimum Spanning Tree
    """
    parent = list(range(n))
    rank = [0] * n
    
    # Find set of vertex with compression if specified
    def find(x):
        if compress:
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]
        else:
            if parent[x] != x:
                return find(parent[x])
            return x

    
    # Union sets by rank
    def union(x, y):
        root_x = find(x)
        root_y = find(y)
        
        if root_x == root_y:
            return
        
        if rank[root_x] < rank[root_y]:
            parent[root_x] = root_y
        else:
            parent[root_y] = root_x
            if rank[root_x] == rank[root_y]:
                rank[root_x] += 1
    
    mst_edges = []
    mst_weight = 0

    # Perform either BucketSort or QuickSort as specified
    if Bucket:
        BucketSort(edges, m)
    else:
        QuickSort(edges, 0, n - 1)
    
    for weight, u, v in edges:
        if find(u) != find(v):  # If including this edge doesn't form a cycle
            union(u, v)  # Include it in the MST
            mst_edges.append({(u, v): weight})
            mst_weight += weight
            
            # MST will have n-1 edges
            if len(mst_edges) == n - 1:
                break
    
    return mst_edges, mst_weight

# Experimentation

First, let's test the basic functionality of our algorithm on a graph with $n = 5$ and a $m$ that is $\Theta(n)$ as a proof of concept.

In [43]:
# Define graph parameters
n = 5
max_weight = 999

# Generate connected graph
A = generate_connected_graph(n, n, 999)
print(f"Pre-Sorted Edges of Graph: {A}")

# Visualize sorting A using our BuckeSort Algorithm
BucketSort(A, max_weight)
print(f"Sorted Edges of Graph: {A}")


Pre-Sorted Edges of Graph: [(643, 0, 1), (481, 1, 2), (935, 0, 3), (231, 1, 4), (347, 4, 2), (561, 0, 4), (910, 3, 4), (198, 3, 1), (552, 2, 3)]
Sorted Edges of Graph: [(198, 3, 1), (231, 1, 4), (347, 4, 2), (481, 1, 2), (552, 2, 3), (561, 0, 4), (643, 0, 1), (910, 3, 4), (935, 0, 3)]


Between these two steps it is clear that our BucketSort is functioning properly, and has sorted the edges in increasing order of edge weight. Let's continue on to the Kruskal's implementation.

In [ ]:
# Run Kruskal's Algorithm
MST, MST_weight = KruskalsAlgorithm(A, n, max_weight)

# View the Results
print(f"MST contains {len(MST)} edges with total weight: {MST_weight}")
for i in range(len(MST)):
    print(f"Edge {i+1}: {MST[i]}")

MST contains 4 edges with total weight: 1337
Edge 1: {(3, 1): 198}
Edge 2: {(1, 4): 231}
Edge 3: {(4, 2): 347}
Edge 4: {(0, 4): 561}


From this simplified implementation, we can see that each vertex has been reached and that minimum number of edges required to reach each vertex ($n - 1$) has ben maintained. Thus it seems as if our algorithm is working properly. Time to move on to explore how path compression can improve runtime. Let's first test how the algorithm performs on a dense graph with $n = 100$ where $m$ is $\Theta(n^2)$. 

In [80]:
# Define graph parameters
n = 1000
max_weight = 999

# Generate connected graph
A = generate_connected_graph(n, 1000, max_weight)

Now that we have a large graph to work with, let's do runtime analysis by performing Kruskal's Algorithm multiple times across our graph and averaging the runtime between iterations.

In [81]:
import time
import numpy as np

# iterables
n_test = 5
runs = []

for _ in range(n_test):

    # Keep track of starting time
    start = time.time()

    # Run Kruskal's Algorithm with optional argument compress set to False
    MST, MST_weight = KruskalsAlgorithm(A, n, max_weight, compress = False)

    # Record the finish time and calculate the total runtime
    runs.append(time.time() - start)

print(f"Kruskal's algorithm without path compression took on average {np.mean(runs)} seconds to run.")

Kruskal's algorithm without path compression took on average 0.00501561164855957 seconds to run.


Let's now run the same code for a Kruskal's implementation which performs path compression.

In [82]:
import time
import numpy as np

# iterables
n_test = 5
runs2 = []

for _ in range(n_test):

    # Keep track of starting time
    start = time.time()

    # Run Kruskal's Algorithm
    MST, MST_weight = KruskalsAlgorithm(A, n, max_weight)

    # Record the finish time and calculate the total runtime
    runs2.append(time.time() - start)

print(f"Kruskal's algorithm with path compression took on average {np.mean(runs2)} seconds to run.")

Kruskal's algorithm with path compression took on average 0.012955236434936523 seconds to run.


Time to compare the runtime of Kruskal's with and without BucketSort on graphs of increasing size.

In [ ]:
# Define graph parameters
n = 100
max_weight = 999

# Generate connected graph
A = generate_connected_graph(n, max_weight)

[(845, 0, 1),
 (615, 1, 2),
 (691, 2, 3),
 (672, 3, 4),
 (679, 3, 5),
 (102, 0, 6),
 (702, 5, 7),
 (924, 6, 8),
 (956, 8, 9),
 (288, 7, 10),
 (565, 2, 11),
 (646, 0, 12),
 (370, 5, 13),
 (327, 8, 14),
 (379, 9, 15),
 (92, 11, 16),
 (971, 10, 17),
 (198, 7, 18),
 (787, 8, 19),
 (888, 17, 20),
 (145, 18, 21),
 (163, 4, 22),
 (329, 15, 23),
 (138, 23, 24),
 (558, 10, 25),
 (719, 16, 26),
 (182, 5, 27),
 (522, 1, 28),
 (167, 23, 29),
 (627, 0, 30),
 (392, 13, 31),
 (870, 21, 32),
 (845, 23, 33),
 (291, 25, 34),
 (658, 26, 35),
 (407, 20, 36),
 (829, 26, 37),
 (892, 30, 38),
 (50, 8, 39),
 (567, 36, 40),
 (559, 39, 41),
 (481, 33, 42),
 (20, 14, 43),
 (557, 3, 44),
 (719, 19, 45),
 (639, 10, 46),
 (106, 43, 47),
 (562, 24, 48),
 (160, 31, 49),
 (698, 45, 50),
 (850, 41, 51),
 (803, 41, 52),
 (779, 5, 53),
 (405, 39, 54),
 (759, 22, 55),
 (569, 35, 56),
 (994, 56, 57),
 (367, 57, 58),
 (338, 49, 59),
 (8, 27, 60),
 (859, 33, 61),
 (262, 49, 62),
 (503, 8, 63),
 (778, 58, 64),
 (657, 47, 65),

In [34]:
# iterables
n_test = 5
runs = []

for _ in range(n_test):

    # Keep track of starting time
    start = time.time()

    # Run Kruskal's Algorithm with optional argument Bucket ss set to False
    MST, MST_weight = KruskalsAlgorithm(A, n, max_weight, Bucket = False)

    # Record the finish time and calculate the total runtime
    runs.append(time.time() - start)

print(f"Kruskal's algorithm with QuickSort took on average {np.mean(runs)} seconds to run.")

Kruskal's algorithm with QuickSort took on average 0.025359439849853515 seconds to run.


In [36]:
# iterables
n_test = 5
runs = []

for _ in range(n_test):

    # Keep track of starting time
    start = time.time()

    # Run Kruskal's Algorithm with BucketSort
    MST, MST_weight = KruskalsAlgorithm(A, n, max_weight, Bucket = True)

    # Record the finish time and calculate the total runtime
    runs.append(time.time() - start)

print(f"Kruskal's algorithm with BucketSort took on average {np.mean(runs)} seconds to run.")

Kruskal's algorithm with BucketSort took on average 0.0036000728607177733 seconds to run.
